# MODULE 1 : RENDER MULTIVIEW IMAGES FROM CAD MODEL

Step 1. Import the CNOS repo
```
!git clone https://github.com/nv-nguyen/cnos.git
```

In [ ]:
!git clone https://github.com/nv-nguyen/cnos.git

Step 2. Install Dependancies
```
!pip install pyrender trimesh Pillow tqdm numpy matplotlib
```

In [ ]:
!pip install pyrender trimesh Pillow tqdm numpy matplotlib

Step 3. Changes in CNOS script
*   /content/cnos/src/utils/trimesh_utils.py ; `get_obj_diameter` function, argument passed is mesh itself not path, so no need to load mesh(change argument passed from 'mesh_path' to 'mesh' and remove the `load_mesh` statement)
*   /content/cnos/src/poses/pyrender.py ; rename to something else :
```
!mv /content/cnos/src/poses/pyrender.py /content/cnos/src/poses/generate_views.py
```




In [3]:
!mv /content/cnos/src/poses/pyrender.py /content/cnos/src/poses/generate_views.py

Step 4. Changes in `generate_views.py` script
*   At the beginning of /content/cnos/src/poses/generate_views.py, type this
```
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.path.dirname(__file__), "../..")))


Step 5. Before running we make a directory to store our model and our rendered images

```
!mkdir -p output/renders
```


In [6]:
!mkdir -p output/renders


UPLOAD YOUR CAD(.PLY) MODEL UNDER THE OUTPUT DIRECTORY (name as `model.ply` for convenience with rest of the code)

or run the script below for an existing model

In [ ]:
!wget -O output/model.ply "https://github.com/pitcher69/IITISOC/raw/refs/heads/main/obj_000000.ply"


VISUALIZE MODEL

In [ ]:
import trimesh

mesh = trimesh.load("output/model.ply")
print("Center (centroid):", mesh.centroid)
mesh.show()

Centre the model to origin if it isnt

In [ ]:
import trimesh
import numpy as np

def normalize_and_overwrite_model(model_path):
    # Load mesh
    mesh = trimesh.load_mesh(model_path)
    print(f"Original center: {mesh.bounding_box.centroid}")

    # Center at origin
    centroid = mesh.bounding_box.centroid
    mesh.apply_translation(-centroid)

    # Uniform scale to unit cube
    max_extent = np.max(mesh.bounding_box.extents)
    if max_extent == 0:
        raise ValueError("Mesh has zero size.")
    mesh.apply_scale(1.0 / max_extent)

    # Optional: set visible color if missing
    if mesh.visual.kind != 'vertex':
        mesh.visual.vertex_colors = np.tile([180, 180, 180, 255], (len(mesh.vertices), 1))

    # Save over the original file
    mesh.export(model_path)
    print(f"New center: {mesh.bounding_box.centroid}")
    print(f"Normalized and saved: {model_path}")

# Example usage
normalize_and_overwrite_model("/content/output/model.ply")


Step 6. Run the rendering script


```
!python path_to_rendering_script(generate_views.py)
path_to_model
path_to_predefined_object_pose(cnos->src->poses->predefined_poses->pick_any_OBJ_pose [level0->42,level1->162,level2->642]
path_to_renders_folder_to_store
0(gpu)
False
0.8(light_intensity)
1.7(radius)

```



In [ ]:
!python /content/cnos/src/poses/generate_views.py /content/output/model.ply /content/cnos/src/poses/predefined_poses/obj_poses_level0.npy /content/output/renders 0 False 0.8 1.7

Generate GIF to visualize the rendered images

In [ ]:
from PIL import Image, ImageSequence
import glob

image_folder = "/content/output/renders"
image_files = sorted(glob.glob(f"{image_folder}/*.png"))

# Load and convert each image to RGB (removes transparency)
frames = []
for img_path in image_files:
    img = Image.open(img_path).convert("RGBA")
    # Fill transparent background with black
    bg = Image.new("RGB", img.size, (0, 0, 0))
    bg.paste(img, mask=img.split()[3])  # Use alpha channel as mask
    frames.append(bg)

# Save as animated GIF
if frames:
    output_path = "/content/output/rendered_views.gif"
    frames[0].save(output_path, format="GIF", save_all=True,
                   append_images=frames[1:], duration=150, loop=0)
    print(f"GIF saved at: {output_path}")
else:
    print("No images found to create GIF.")


# MODULE 2A: EXTRACT FEATURES FROM RENDERED IMAGES (2D FEATURE VECTOR)

Using DINOv2 we extract features of rendered images

In [21]:
import glob
import os
from PIL import Image
from transformers import AutoImageProcessor, AutoModel
import torch

# Load processor and model
print("Loading model and processor...")
processor = AutoImageProcessor.from_pretrained("facebook/dinov2-base")
model = AutoModel.from_pretrained("facebook/dinov2-base").eval()
print("Loaded model and processor.")

# Load all render image files
render_files = sorted(glob.glob("output/renders/*.png"))
print(f"Found {len(render_files)} images")

all_features = []

for i, file in enumerate(render_files):
    if not os.path.isfile(file):
        print(f"⚠️ Skipping non-file: {file}")
        continue

    try:
        img = Image.open(file).convert("RGB")
    except Exception as e:
        print(f"Could not open image {file}: {e}")
        continue

    inputs = processor(images=img, return_tensors="pt")

    with torch.no_grad():
        output = model(**inputs)
        feats = output.last_hidden_state  # [1, tokens, dim]

    if feats is not None and feats.numel() > 0:
        all_features.append(feats[0])  # shape: [tokens, dim]
    else:
        print(f"Empty features for image {i}: {file}")

# Final stacking
if not all_features:
    raise ValueError("No features extracted. Please check your images or rendering.")
else:
    all_features_tensor = torch.stack(all_features, dim=0)
    print(f"Final feature tensor shape: {all_features_tensor.shape}")
    for i in range(len(all_features_tensor)):
      print(f"Feature tensor {i}: {all_features_tensor[i][0][:5]}")


Loading model and processor...
Loaded model and processor.
Found 42 images
Final feature tensor shape: torch.Size([42, 257, 768])
Feature tensor 0: tensor([ 2.2824, -1.0590, -0.0379,  0.1175, -2.7167])
Feature tensor 1: tensor([ 0.5527, -0.4434,  1.3652,  3.0403,  1.0943])
Feature tensor 2: tensor([ 1.2104,  0.1871,  0.5861, -1.6176, -0.7504])
Feature tensor 3: tensor([ 0.5081, -1.6777,  0.2116, -0.0277, -2.0972])
Feature tensor 4: tensor([ 1.6865,  0.6217,  0.8571, -0.5999, -2.0089])
Feature tensor 5: tensor([ 1.1142,  1.1273,  1.0218,  1.5027, -1.4039])
Feature tensor 6: tensor([-0.1990, -1.0128,  0.9661, -1.4182, -1.0658])
Feature tensor 7: tensor([ 1.3545, -1.2037,  1.5297, -0.7482, -1.4338])
Feature tensor 8: tensor([ 0.2041,  0.2962,  0.5079, -0.8700, -1.2814])
Feature tensor 9: tensor([ 0.4600, -0.8905,  0.8067, -0.0425, -2.1945])
Feature tensor 10: tensor([ 1.9138,  0.8016,  1.1402, -2.0447, -2.2197])
Feature tensor 11: tensor([ 2.2335, -0.4293,  1.7023, -0.3206, -0.7663])
Feat